In [3]:
import numpy as np
import tensorflow as tf
import cv2 as cv2
from tensorflow.keras import models, layers 
from tensorflow.keras.utils import to_categorical 
import numpy as np

In [2]:
with open('models/object_detection_classes_coco.txt', 'r') as f:
    class_names = f.read().split('\n')

# get a different color array for each of the classes
COLORS = np.random.uniform(0, 255, size=(len(class_names), 3))

# load the DNN model
mobile_net_model = cv2.dnn.readNet(model='models/frozen_inference_graph.pb',
                        config='models/ssd_mobilenet_v2_coco_2018_03_29.pbtxt.txt', 
                        framework='TensorFlow')

In [10]:
def getFrameWithBoxes(img):
    copy_image = img.copy()
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    image_height, image_width, _ = image.shape
    # create blob from image
    blob = cv2.dnn.blobFromImage(image=image, size=(300, 300), mean=(104, 117, 123))
    # create blob from image
    mobile_net_model.setInput(blob)
    # forward pass through the mobile_net_model to carry out the detection
    output = mobile_net_model.forward()

    # loop over each of the detection
    for detection in output[0, 0, :, :]:
        # extract the confidence of the detection
        confidence = detection[2]
        # draw bounding boxes only if the detection confidence is above...
        # ... a certain threshold, else skip
        if confidence > .7:
            # get the class id
            class_id = detection[1]
            # map the class id to the class
            class_name = class_names[int(class_id)-1]
            color = COLORS[int(class_id)]
            # get the bounding box coordinates
            box_x = detection[3] * image_width
            box_y = detection[4] * image_height
            # get the bounding box width and height
            box_width = detection[5] * image_width
            box_height = detection[6] * image_height
            # draw a rectangle around each detected object
            cv2.rectangle(copy_image, (int(box_x), int(box_y)), (int(box_width), int(box_height)), color, thickness=2)
            # put the FPS text on top of the frame
            cv2.putText(copy_image, class_name, (int(box_x), int(box_y - 5)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    return copy_image

In [5]:
cap = cv2.VideoCapture('input/video_1.mp4')

# get the video frames' width and height for proper saving of videos
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# create the `VideoWriter()` object
out = cv2.VideoWriter('output/video_result_1.mp4',
                      cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))


In [6]:
def equalize_RGB(img):
    # convert to YUV color space
    red = img[:,:,0]
    green = img[:,:,1]
    blue = img[:,:,2]
    # equalize the red, green, and blue channels
    red_eq = cv2.equalizeHist(red)
    green_eq = cv2.equalizeHist(green)
    blue_eq = cv2.equalizeHist(blue)
    # merge the channels back together
    img_eq = cv2.merge((red_eq, green_eq, blue_eq))
    return img_eq

In [11]:
# detect objects in each frame of the video
while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        image_eq = equalize_RGB(frame)
        image_boxes = getFrameWithBoxes(image_eq)

        cv2.imshow('image', image_boxes)
        out.write(image_boxes)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    else:
        break

cap.release()
cv2.destroyAllWindows()

: 